# Music Genre Classification

## Recurrent Neural Network Implementation

Fabrizio Niro - Jacopo Signò

GTZAN Dataset - Music Genre Classification 

https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification

In [64]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
from torchinfo import summary
import pickle
import numpy as np

In [33]:
with open('Processed_data/RNN/X_train', 'rb') as f: 
    X_train = pickle.load(f)

with open('Processed_data/RNN/y_train', 'rb') as f: 
    y_train = pickle.load(f)

with open('Processed_data/RNN/X_test', 'rb') as f: 
    X_test = pickle.load(f)

with open('Processed_data/RNN/y_test', 'rb') as f: 
    y_test = pickle.load(f)

In [49]:
X_train  = Variable(torch.from_numpy(X_train)).float()
X_test  = Variable(torch.from_numpy(X_test)).float()

In [50]:
type(X_train)

torch.Tensor

In [51]:
type(y_train)

torch.Tensor

In [53]:
type(X_test)

torch.Tensor

In [52]:
type(y_test)

torch.Tensor

In [54]:
device = "cuda" if torch.cuda.is_available() else "cpu"

X_train = X_train.to(device)
y_train = y_train.to(device)

X_test = X_test.to(device)
y_test = y_test.to(device)

In [55]:
X_train

tensor([[ 0.6919,  0.6092,  0.4423,  ..., -0.7881, -0.4091,  0.6552],
        [ 1.0082,  0.9300,  0.8769,  ...,  1.0314,  1.1990,  0.9908],
        [ 0.6341,  0.8203,  0.9406,  ..., -0.8749, -0.5790, -0.2922],
        ...,
        [-0.8810, -0.8742, -0.8877,  ..., -0.0589,  0.2270,  0.5274],
        [-0.5435, -0.4778, -0.5571,  ...,  1.4441,  1.4098,  1.5974],
        [-0.6507, -0.8143, -0.9505,  ...,  0.0628, -0.2014, -0.2742]],
       device='cuda:0')

In [56]:
y_train

tensor([0, 7, 2,  ..., 7, 4, 8], device='cuda:0')

In [57]:
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(50, 3)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x

In [58]:
print(f"Using {device} device")

model     = Model(X_train.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn   = nn.CrossEntropyLoss()
print(model)

Using cuda device
Model(
  (layer1): Linear(in_features=2580, out_features=50, bias=True)
  (layer2): Linear(in_features=50, out_features=50, bias=True)
  (layer3): Linear(in_features=50, out_features=3, bias=True)
)


In [65]:
summary(model)

Layer (type:depth-idx)                   Param #
Model                                    --
├─Linear: 1-1                            129,050
├─Linear: 1-2                            2,550
├─Linear: 1-3                            153
Total params: 131,753
Trainable params: 131,753
Non-trainable params: 0

In [60]:
import tqdm

EPOCHS  = 250
#X_train = Variable(torch.from_numpy(X_train)).float()
#y_train = Variable(torch.from_numpy(y_train)).long()
#X_test  = Variable(torch.from_numpy(X_test)).float()
#y_test  = Variable(torch.from_numpy(y_test)).long()

loss_list     = np.zeros((EPOCHS,))
accuracy_list = np.zeros((EPOCHS,))

for epoch in tqdm.trange(EPOCHS):
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    loss_list[epoch] = loss.item()
    
    # Zero gradients
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        y_pred = model(X_test)
        correct = (torch.argmax(y_pred, dim=1) == y_test).type(torch.FloatTensor)
        accuracy_list[epoch] = correct.mean()
        print("epoch ["+str(epoch)+"] - accuracy: "+ str(accuracy_list[epoch]) )

  0%|          | 0/250 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(12, 6), sharex=True)

ax1.plot(accuracy_list)
ax1.set_ylabel("validation accuracy")
ax2.plot(loss_list)
ax2.set_ylabel("validation loss")
ax2.set_xlabel("epochs");